# Face Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensein/senselab/blob/main/tutorials/video/face_analysis.ipynb)

This tutorial demonstrates how to use Senselab's Face Analysis API for analyzing human faces. At the moment, this API supports available analysis through [DeepFace](https://github.com/serengil/deepface).

## Setup

Let's get started by installing Senselab and importing the necessary modules from Senselab for processing images/videos and performing face analysis.

In [ ]:
%pip install senselab['video']

In [ ]:
import pprint
import math

from IPython.display import Image, display
from IPython.display import Video as IPyVideo

from senselab.video.data_structures.video import Video
from senselab.video.tasks.face_analysis import (
    analyze_face_attributes,
    extract_face_embeddings,
    recognize_faces,
    verify_faces,
)

In [ ]:
!mkdir -p tutorial_images && mkdir -p tutorial_images/db
!wget -O tutorial_images/sally_1.jpg https://raw.githubusercontent.com/sensein/senselab/main/src/tests/data_for_testing/face_data/sally_1.jpg
!wget -O tutorial_images/sally_vid.mp4 https://raw.githubusercontent.com/sensein/senselab/main/src/tests/data_for_testing/face_data/sally_vid.mp4
!wget -O tutorial_images/db/group.jpg https://raw.githubusercontent.com/sensein/senselab/main/src/tests/data_for_testing/face_data/db/group.jpg
!wget -O tutorial_images/db/sally_2.jpg https://raw.githubusercontent.com/sensein/senselab/main/src/tests/data_for_testing/face_data/db/sally_2.jpg

In [ ]:
TUTORIAL_PATH = "tutorial_images"
DB_PATH = TUTORIAL_PATH + "/db"

SALLY_1 = TUTORIAL_PATH + "/sally_1.jpg"
SALLY_2 = DB_PATH + "/sally_2.jpg"
GROUP = DB_PATH + "/group.jpg"

SALLY_VID_PATH = TUTORIAL_PATH + "/sally_vid.mp4"
SALLY_VID = Video.from_filepath(str(SALLY_VID_PATH))
frame_sample_rate = 2.0 # Sample 2 frames per second

## Preview Inputs

Let’s display the input media we’ll use for the tutorial:

In [ ]:
# Show images
print("SALLY_1 (Query Image)")
display(Image(SALLY_1, width=300))

print("SALLY_2 (Database Image)")
display(Image(SALLY_2, width=300))

print("GROUP (Database Image)")
display(Image(GROUP, width=300))

print("SALLY_VID (Query Video)")
display(IPyVideo(SALLY_VID_PATH, embed=True, width=500))

## Understand Outputs

Senselab's face analysis functions return a list of DetectedFace objects for each frame. Each DetectedFace can contain:

- **bbox**: Bounding box of the detected face

- **frame_ix**: Index of the frame the face was found in (for video)

- **face_confidence**: Confidence score of the face detector

- **face_match**: A list of FaceMatch objects for face recognition or verification

- **embedding**: A vector representation of the face

- **attributes**: Estimated facial attributes (age, gender, emotion, race)

These fields vary based on the specific analysis method you use.

## Recognize Faces

### In an image
Let's begin by checking if individuals in a particular image can be matched with any indivduals in a database of images (in this context a folder containing different images).

In [ ]:
recognized_img = recognize_faces(SALLY_1, DB_PATH)

In [ ]:
print("Image Face Recognition Summary:")

if recognized_img and recognized_img[0]:
    frame = recognized_img[0] # Since this is a single image, we expect 1 frame
    print(f"{len(frame)} face(s) detected")
    for face_ix, face in enumerate(frame):
        print(f"  Face {face_ix + 1} matches:")
        for match in face.face_match:
            print(f"    ID: {match.identity} | Distance: {round(match.distance, 4)} | Verified: {match.verified}")
    print("")

### In a video
Next, let's compare faces in a video by frame with any indivdual in a database of images (in this context a folder containing different images).

Setting `enforce_detection=False` ensures that an error is not thrown if if the model cannot identify a face in a given frame.

In [ ]:
recognized_vid = recognize_faces(SALLY_VID, DB_PATH, frame_sample_rate=frame_sample_rate, enforce_detection=False)

In [ ]:
print(f"Video Embedding Frames: {len(recognized_vid)}") # Based on frame_sample_rate (set frame_sample_rate=None to process all frames)

for frame in recognized_vid:
    if not frame:
        continue
    print(f"Frame {frame[0].frame_ix} - {len(frame)} face(s) detected")
    for face_ix, face in enumerate(frame):
        print(f"  Face {face_ix + 1} matches:")
        for match in face.face_match:
            print(f"    ID: {match.identity} | Distance: {round(match.distance, 4)} | Verified: {match.verified}")
    print("")

: 

# Verify Faces

Here, we can compare 2 images to confirm whether the same person is in both. 

Note: This currently only works with images containing a single individual. If working with multi-individual images or videos, use the `recognize_faces` function.

In [ ]:
verify = verify_faces(SALLY_1, SALLY_2)

# We expect verified to be true
pprint.pprint(verify.face_match[0].verified)

## Extract Embeddings

This function returns a numerical representation of the face that can be used for similarity comparison, clustering, or custom analysis.

In [ ]:
img_embeddings = extract_face_embeddings(GROUP)

In [ ]:
if img_embeddings and img_embeddings[0]:
    embeddings = img_embeddings[0] # Since this is a single image, we expect 1 frame
    print("Printing first 5 values of each face embedding.\n")
    print(f"{len(embeddings)} face(s) detected")
    for face_ix, face in enumerate(embeddings):
        print(f"  Face {face_ix + 1} Embedding {face.embedding[:5]}")

### Embeddings from video

You can also extract embeddings frame-by-frame from a video:

In [ ]:
video_embeddings = extract_face_embeddings(SALLY_VID, frame_sample_rate=frame_sample_rate, enforce_detection=False)

In [ ]:
print(f"Video Embedding Frames: {len(video_embeddings)}")
print("Printing first 5 values of each face embedding.\n")
for frame in video_embeddings:
    if not frame:
        continue
    print(f"Frame {frame[0].frame_ix} - {len(frame)} face(s) detected")
    for face_ix, face in enumerate(frame):
        print(f"  Face {face_ix + 1} Embedding: {face.embedding[:5]}")
    print("")

## Analyze Face Attributes

This function analyzes and returns facial attributes such as age, emotion, gender, and race.

In [ ]:
img_attributes = analyze_face_attributes(SALLY_1, actions=["age", "gender", "emotion", "race"], 
                                        detector_backend="retinaface")

In [ ]:
if img_attributes and img_attributes[0]:
    frame = img_attributes[0] # Since this is a single image, we expect 1 frame
    print(f"{len(frame)} face(s) detected")
    for face_ix, face in enumerate(frame):
        print(f"  Face {face_ix + 1} attributes:")
        attribute = face.attributes
        print(f"    Age: {attribute.age} | Gender: {attribute.dominant_gender} | Emotion: {attribute.dominant_emotion} | Race: {attribute.dominant_race}")
    print("")

### Attributes from video

Attributes can also be extracted from videos:

In [ ]:
video_attributes = analyze_face_attributes(SALLY_VID, actions=["age", "gender"], frame_sample_rate=frame_sample_rate, enforce_detection=False, detector_backend="retinaface")

In [ ]:
for frame_ix, frame in enumerate(video_attributes):
    for face_ix, face in enumerate(frame):
        if face_ix==0:
            print(f"Frame {face.frame_ix} - Faces detected: {len(frame)}")
        print(f"  Face {face_ix + 1}:")
        attribute = face.attributes
        print(f"    Age: {attribute.age} | Gender: {attribute.dominant_gender}")
    print("")

In this tutorial, we explored how to use Senselab’s face analysis API for:

- Recognizing individuals in images and videos
- Verifying whether two images represent the same person
- Extracting numerical embeddings of faces in images and videos
- Analyzing age, gender, race, and emotion in images and videos

See the [Senselab Documentation](https://sensein.group/senselab/senselab.html) for function-specific parameters such as recognition model, alignment settings, threshold, detector backend, and more.